### 특성 선택 방식

- 필터(Filter) : 통계적인 속성을 조사하여 가장 뛰어난 특성 선택
- 래퍼(Wrapper) : 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합 선택
- 임베디드(Embedded) : 학습 알고리즘의 훈련 단계를 확장하거나 일부로 구성하여 가장 좋은 특성의 부분 조합 선택

## 분산을 기준으로 수치 특성 선택하기
#### 수치형 특서 중에서 분산이 낮은 특성(== 정보가 거의 없는 특성)을 삭제

In [2]:
# 주어진 기준값보다 높은 분산을 가진 특성을 선택한다

from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

In [3]:
iris=datasets.load_iris()

In [4]:
features=iris.data
target=iris.target

In [5]:
# 기준값 생성
thresholder=VarianceThreshold(threshold=.5)

In [6]:
# 기준값보다 높은 특성 선택

feature_high_variance=thresholder.fit_transform(features)

In [7]:
feature_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [9]:
# 분산확인

thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

** 주의할점

1. 분산은 원점에 맞춰진 값이 아닌 특성의 제곱단위이다. 따라서 특성의 단위가 서로 다르면 VT는 작동하지 않는다.
2. 분산의 기준값을 수동으로 선택(threshold 지정) 하기 때문에 어떤 값이 좋은지 판단할 수 있어야 한다.
3. 특성이 표준화(평균=0, 단위분산)되어 있으면 분산 기준 선택 방식은 올바르게 작동하지 않는다.

In [10]:
# 3번 주의할점 확인

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
features_std=scaler.fit_transform(features)

In [11]:
selector=VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## 분산을 기준으로 이진 특성 선택하기
### 이진 범주형 특성에서 분산이 낮은 특성(적은 정보를 가진 특성)을 삭제

In [13]:
# 베르누이 확률 변수의 분산이 기준값 이상인 특성을 선택

from sklearn.feature_selection import VarianceThreshold

In [14]:
# 예제 특성 행렬
# 특성 0 : 80%가 클래스 0
# 특성 1 : 80%가 클래스 1
# 특성 2 : 60%가 클래스 0, 40%는 클래스 1

features=[[0,1,0],
         [0,1,1],
         [0,1,0],
         [0,1,1],
          [1,0,0]]

In [16]:
# 분산을 기준으로 선택
thresholder=VarianceThreshold(threshold=(.75 * (1-.75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

In [17]:
# 분산 확인

thresholder.variances_

array([0.16, 0.16, 0.24])

In [18]:
'''
VarainceThreshold 클래스는 수치 특성의 이진 특성에 상관없이
넘파이 var함수를 사용하여 분산을 계산한다.'''

import numpy as np

np.var(features, axis=0)

array([0.16, 0.16, 0.24])

## 상관관계가 큰 특성 다루기

In [19]:
import pandas as pd
import numpy as np

C:\Users\river\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [20]:
#상관관계가 큰 특성행렬 만들기

features=np.array([[1,1,1],
                  [2,2,0,],
                   [3,3,1],
                   [4,4,0],
                   [5,5,1],
                   [6,6,0],
                   [7,7,1],
                   [8,7,0],
                   [9,7,1]])

In [21]:
dataframe=pd.DataFrame(features)

In [23]:
# 상관관계 행렬 만들기
corr_matrix=dataframe.corr().abs()
corr_matrix

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,0.034503
2,0.000000,0.034503,1.000000


In [26]:
# 상관관계 행렬의 상삼각 행렬을 선택

upper=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [28]:
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다

to_drop=[column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

[1]

In [29]:
# 특성 삭제

dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [30]:
# 상관관계 행렬은 넘파이 corrcoef 함수로 구할 수도 있다.
# 이 함수는 특성이 행에 놓여 있을 것으로 기대한다.
# 특성이 열에 놓여 있다고 알려주려면 rowvar 매개변수를 False

np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [31]:
'''
해결에 나온 np.triu 함수는 주어진 배열에서 상삼각 행렬을 추출하여 반환한다.
매개변수 k가 기본값이 0이면 반환되는 행렬에 대각원소가 포함된다.
k 값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환한다.
'''

# ex) k=2

np.triu(np.ones((4,4)),k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [32]:
# 하삼각 행렬을 구하는 방법

np.tril(np.ones((4,4)),k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

## 분류 작업에 관련 없는 특성 삭제하기
### 범주형/수치형 타깃 벡터에서 관련 없는 특성을 삭제하기

In [33]:
# 범주형 특성은 각 특성과 타깃 벡터 사이의 카이제곱 통계를 계산

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

In [35]:
# 데이터 로드

iris=load_iris()
features=iris.data
target=iris.target

features[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [40]:
# 범주형 데이터를 정수형으로 변환
features=features.astype(int)
features[0:3]

array([[5, 3, 1, 0],
       [4, 3, 1, 0],
       [4, 3, 1, 0]])

In [51]:
# 카이제곱 통계값이 가장 큰 특성 두개 선택
chi2_selector=SelectKBest(chi2, k=2) # k : 선택하려는 특성의 개수  결정
features_kbest=chi2_selector.fit_transform(features, target)

In [52]:
# 결과확인

print('원본 특성 개수: ',features.shape[1])
print('줄어든 특성 개수: ',features_kbest.shape[1])

원본 특성 개수:  4
줄어든 특성 개수:  2


특성이 수치형이면 각 특성과 타깃 벡터 사이에서 분산분석(ANOVA)의 F값을 계산한다

In [53]:
# F값이 가장 높은 특성 두개를 선택한다

fvalue_selector=SelectKBest(f_classif, k=2)
features_kbest=fvalue_selector.fit_transform(features, target)

In [54]:
print('원본 특성 개수: ',features.shape[1])
print('줄어든 특성 개수: ',features_kbest.shape[1])

원본 특성 개수:  4
줄어든 특성 개수:  2


특정 특성 개수를 선택하는 대신 SelectPercentile을 사용하여 특성의 상위 n퍼센트를 선택할 수 있다

In [55]:
from sklearn.feature_selection import SelectPercentile

In [56]:
# 가장 큰 F값의 상위 75% 특성을 선택한다

fvalue_selector=SelectPercentile(f_classif, percentile=75)
features_kbest=fvalue_selector.fit_transform(features, target)

In [57]:
print('원본 특성 개수 : ',features.shape[1])
print('줄어든 특성 개수 : ',features_kbest.shape[1])

원본 특성 개수 :  4
줄어든 특성 개수 :  3


### 카이제곱 통계

- 두 범주형 벡터의 독립성 평가
- 카이제곱 특성은 관찰 빈도와 전혀 관계가 없다고 기대하는 빈도 사이에 얼마나 큰 차이가 있는지 알려주는 하나의 숫자이다.
- 카이제곱 통계를 계산하면 특성과 타깃 벡터 사이의 독립성을 측정할 수 있다.
- 특성 변수가 타깃에 독립적이면 분류 문제에 사용할 정보가 없기 때문에 목적에 맞지 않다.
- 반면에 두 변수가 크게 의존적이면 모델 훈련에 필요한 정보가 많다.
- 카이제곱 방식을 사용하려면 수치형 특성을 범주형 특성으로 변환해야 한다.
- 카이제곱 방식을 사용하려면 모든 값이 음수가 아니어야 한다.

### 분산분석의 F값 통계

- 타깃 벡터로 수치형 특성을 그룹핑하여 각 그룹의 평균이 크게 차이나는지 평가

In [66]:
'''
카이제곱 통계 절차 밟아보기
'''

# 붓꽃 데이터셋 타깃 데이터

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [67]:
# 관찰 빈도 구하기

observed=np.sum(features.reshape(3,50,4),axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [68]:
# 특성과 타깃이 관계가 없다면, 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 된다.

expected=features.sum(axis=0) /3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [69]:
# 카이제곱 공식에 기대빈도와 관찰빈도를 대입한다

np.sum((observed-expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

위의 값중에서 카이제곱 값이 큰 세번째, 네번째 특성이 k=2로 설정했을 때의
두 개의 특성으로 설정된다. 이 값은 chi2_selector.scores_로 확인할 수 있다.

In [70]:
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [71]:
'''
F값 통계 절차 밟아보기
'''

total_mean=np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [72]:
class_mean=np.mean(features.reshape(3,50,4),axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [73]:
# SS_BETWEEN

ss_between=np.sum(50*(class_mean-total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

In [74]:
# SS_TOTAL

ss_total=np.sum((features-total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [76]:
# F값 공식 대입   k=3 (클래스 개수), n=150 (샘플 개수)

f=(ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [77]:
# scores_ 속성으로 F값 확인하기

fvalue_selector.scores_

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

ANOVA는 각 특성이 독립적으로 평가되기 때문에 일변량 분석이라고도 부른다.
회귀일때는 f_classif  대신 f_regression 함수를 사용한다.

## 재귀적 특성 제거하기
### 자동으로 최선의 특성을 선택하기 (RFECV) - 교차검증 사용

In [79]:
# 모델 성능이 나빠질 때까지 특성을 제거하면서 반복적으로 모델 훈련

from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

In [80]:
features, target = make_regression(n_samples=10000, n_features=100, n_informative=2, random_state=1)

In [81]:
# 선형 회귀모델 만들기

ols=linear_model.LinearRegression()

In [89]:
# 재귀적 특성 제거

# estimator : 훈련에 사용할 모델 객체
# step : 매 반복에서 삭제할 특성의 개수나 비율
# scoring : 모델의 평가지표

rfecv=RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799, -0.28547464,  0.7031277 ],
       [-1.07500204, -0.8689623 ,  2.56148527],
       [ 1.37940721, -0.14714771, -1.77039484],
       ...,
       [-0.80331656, -1.030216  , -1.60648007],
       [ 0.39508844, -0.91553464, -1.34564911],
       [-0.55383035, -0.69804472,  0.82880112]])

In [90]:
# RFE (recursive feature elimination) 을 수행하고 난 후 남은 특성 개수 확인

rfecv.n_features_

3

In [91]:
# 어떤 특성이 선택되었는지 확인

rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [92]:
# 특성 순위 확인

rfecv.ranking_

array([80, 98, 39, 86, 31,  1,  8, 74, 58, 33, 30, 32,  4, 20, 13, 71, 24,
       12, 96, 60, 62, 26, 23,  9, 36, 82, 19, 84, 10, 21, 17, 61,  1, 73,
       16, 14, 65, 63, 11,  1, 40, 92, 37, 91, 94, 85, 29, 15, 67, 57, 66,
       56, 83, 18, 46, 38, 41, 45, 54, 68, 89, 59, 81, 75, 42, 78, 88, 22,
       25,  2, 44, 34,  6, 48, 50,  5, 72, 93, 77, 43, 76, 27,  7, 64, 69,
       70,  3, 97, 87, 90, 47, 53, 95, 49, 28, 55, 52, 51, 35, 79])

In [93]:
'''
교차 검증을 사용하지 않는 재귀적 특성 제거 방법
'''

from sklearn.feature_selection import RFE

rfe=RFE(estimator=ols, n_features_to_select=3) # n_features_to_select : 남길 최소 특성의 개수
rfe.fit(features, target)
rfe.transform(features)

array([[ 0.00850799, -0.28547464,  0.7031277 ],
       [-1.07500204, -0.8689623 ,  2.56148527],
       [ 1.37940721, -0.14714771, -1.77039484],
       ...,
       [-0.80331656, -1.030216  , -1.60648007],
       [ 0.39508844, -0.91553464, -1.34564911],
       [-0.55383035, -0.69804472,  0.82880112]])

In [94]:
'''
rfe 객체가 선택한 특성이 rfecv 객체가 선택한 특성과 동일한지 확인
'''
np.all(rfe.support_ == rfecv.support_)

True